In [26]:
# Processing tools to read from Helsinki WFS API and produce GeoJSON

import json

import geojson
import requests


# Helsinki Service Map data

worship, cultural_facilities and community_centers data is available from Helsinki Service Map API. The API is documented at https://www.hel.fi/palvelukarttaws/restpages/avoindata/en


In [27]:
servicemap_object_sample = {
    "id": 71884,
    "contract_type": {
        "id": "MUNICIPAL_SERVICE",
        "description": {
            "fi": "kunnallinen palvelu, Kulttuurin ja vapaa-ajan toimiala, Helsingin kaupunki",
            "sv": "kommunal tjänst, Kultur- och fritidssektorn, Helsingfors stad",
            "en": "municipal service, Culture and Leisure Division, City of Helsinki"
        }
    },
    "name": {
        "fi": "Kalasataman kirjasto",
        "sv": "Fiskehamnens bibliotek",
        "en": "Kalasatama Library"
    },
    "street_address": {
        "fi": "Hermannin rantatie 5",
        "sv": "Hermanstads strandväg 5",
        "en": "Hermannin rantatie 5"
    },
    "municipality": "helsinki",
    "service_nodes": [
        328,
        349
    ],
    "location": {
        "type": "Point",
        "coordinates": [
            24.978022,
            60.187717
        ]
    },
    "geometry": {
        "type": "Point",
        "coordinates": [
            24.978022,
            60.187717
        ]
    },
    "accessibility_shortcoming_count": {}
}


def convert_servicemap_object_to_geojson(obj):
    """
    Converts a single object from Helsinki Service Map API to GeoJSON.
    See servicemap_object_sample for an example of the input object.
    """
    geojson = {
        "type": "Feature",
        "properties": {
            "id": obj["id"],
            "name": obj["name"],
            "street_address": obj["street_address"],
        },
        "geometry": obj["geometry"]
    }
    return geojson


def get_servicemap_url_and_return_geojson(url: str, contract_type: str = "MUNICIPAL_SERVICE"):
    """
    Gets a list of objects from Helsinki Service Map API and converts them to GeoJSON.
    """
    res = requests.get(url)
    data = res.json()
    feature_list = []
    for obj in data["results"]:
        # print(json.dumps(obj, indent=2))
        if obj["municipality"] == "helsinki" and (contract_type == "" or obj["contract_type"]["id"] == contract_type):
            feature_list.append(convert_servicemap_object_to_geojson(obj))
    return geojson.FeatureCollection(feature_list)


In [29]:
# Helsinki Service Map data and save to GeoJSON files

cultural_facilities_url = "https://api.hel.fi/servicemap/v2/unit/?service=345&page=1&page_size=200&only=street_address%2Cname%2Cmunicipality%2Ccontract_type&geometry=true&language=fi"

geojson_obj = get_servicemap_url_and_return_geojson(cultural_facilities_url)
fname = "HelsinkiCulturalFacilities.json"
with open(fname, "w") as f:
    json.dump(geojson_obj, f)
print(f"* [{fname}](https://proto.fvh.io/urbanage/{fname})")


library_url = "https://api.hel.fi/servicemap/v2/unit/?service=813&page_size=200&geometry=true&only=street_address%2Cname%2Cmunicipality%2Ccontract_type&page=1"

geojson_obj = get_servicemap_url_and_return_geojson(library_url)
fname = "HelsinkiLibraries.json"
with open(fname, "w") as f:
    json.dump(geojson_obj, f)
print(f"* [{fname}](https://proto.fvh.io/urbanage/{fname})")


worship_url = "https://api.hel.fi/servicemap/v2/unit/?service=314&page=1&page_size=200&only=street_address%2Cname%2Cmunicipality%2Ccontract_type&geometry=true&language=fi"

geojson_obj = get_servicemap_url_and_return_geojson(worship_url, "")
fname = "HelsinkiWorships.json"
with open(fname, "w") as f:
    json.dump(geojson_obj, f)
print(f"* [{fname}](https://proto.fvh.io/urbanage/{fname})")


* [HelsinkiCulturalFacilities.geojson](https://proto.fvh.io/urbanage/HelsinkiCulturalFacilities.geojson)
* [HelsinkiLibraries.geojson](https://proto.fvh.io/urbanage/HelsinkiLibraries.geojson)
* [HelsinkiWorships.geojson](https://proto.fvh.io/urbanage/HelsinkiWorships.geojson)
